In [1]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

NVIDIA API Key: ········
Retrieved NVIDIA_API_KEY beginning with "nvapi-ZAj..."


{'playground_kosmos_2': '0bcd1a8c-451f-4b12-b7f0-64b4781190d1',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_gemma_7b': '1361fa56-61d7-4a12-af32-69a3825746fa',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'playground_nemotron_steerlm_8b': '1423ff2f-d1c7-4061-82a7-9e8c67afd43a',
 'playground_phi2': '6251d6d2-54ee-4486-90f4-2792bf0d3acd',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'playground_starcoder2_15b': '6acada03-fe2f-4e4d-9e0a-e711b9fd1b59',
 'playground_sdxl': '89848fb8-549f-41bb-88cb-95d6597044a4',
 'playground_nv_llama2_rlhf_70b': '7b3e3361-4266-41c8-b312-f5e33c81fc92',
 'playground_neva_22b': '8bf70738-59b9-4e5f-bc87-7ab4203be7a0',
 'playground_clip': '8c21289c-0b18-446d-8838-011b7249c513',
 'playground_fuyu_8b': '9f757064-657f-4c85-abd7-37a7a9b6ee11',
 'playgroun

In [ ]:
%pip install --upgrade --quiet  arxiv

In [ ]:
%pip install --upgrade --quiet  pymupdf

## Part 1: Chatting with Documents

In [2]:
%%time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import ArxivLoader

## Loading in the file

## Unstructured File Loader: Good for arbitrary "probably good enough" loader
# documents = UnstructuredFileLoader("llama2_paper.pdf").load()

## More specialized loader, won't work for everything, but simple API and usually better results
documents = ArxivLoader(query="2205.00445").load()  ## MRKL
# documents = ArxivLoader(query="2210.03629").load()  ## ReAct

CPU times: user 367 ms, sys: 499 ms, total: 865 ms
Wall time: 5.91 s


In [3]:
## Printing out a sample of the content
print("Number of Documents Retrieved:", len(documents))
print(f"Sample of Document 1 Content (Total Length: {len(documents[0].page_content)}):")
print(documents[0].page_content[:1000])

Number of Documents Retrieved: 1
Sample of Document 1 Content (Total Length: 36261):
MRKL Systems
A modular, neuro-symbolic architecture that combines large language
models, external knowledge sources and discrete reasoning
Ehud Karpas, Omri Abend, Yonatan Belinkov, Barak Lenz, Opher Lieber,
Nir Ratner, Yoav Shoham, Hoﬁt Bata, Yoav Levine, Kevin Leyton-Brown,
Dor Muhlgay, Noam Rozen, Erez Schwartz, Gal Shachaf, Shai
Shalev-Shwartz, Amnon Shashua, Moshe Tenenholtz
AI21 Labs
May 3, 2022
Abstract
Huge language models (LMs) have ushered in a new era for AI, serving as a gate-
way to natural-language-based knowledge tasks. Although an essential element of
modern AI, LMs are also inherently limited in a number of ways. We discuss these
limitations and how they can be avoided by adopting a systems approach. Con-
ceptualizing the challenge as one that involves knowledge and reasoning in addition
to linguistic processing, we deﬁne a ﬂexible architecture with multiple neural mod-
els, complement

In [4]:
from pprint import pprint
pprint(documents[0].metadata)

{'Authors': 'Ehud Karpas, Omri Abend, Yonatan Belinkov, Barak Lenz, Opher '
            'Lieber, Nir Ratner, Yoav Shoham, Hofit Bata, Yoav Levine, Kevin '
            'Leyton-Brown, Dor Muhlgay, Noam Rozen, Erez Schwartz, Gal '
            'Shachaf, Shai Shalev-Shwartz, Amnon Shashua, Moshe Tenenholtz',
 'Published': '2022-05-01',
 'Summary': 'Huge language models (LMs) have ushered in a new era for AI, '
            'serving as a\n'
            'gateway to natural-language-based knowledge tasks. Although an '
            'essential\n'
            'element of modern AI, LMs are also inherently limited in a number '
            'of ways. We\n'
            'discuss these limitations and how they can be avoided by adopting '
            'a systems\n'
            'approach. Conceptualizing the challenge as one that involves '
            'knowledge and\n'
            'reasoning in addition to linguistic processing, we define a '
            'flexible\n'
            'architecture with multi

## Transforming Documents

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## Some nice custom preprocessing
# documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(documents)

# def include_doc(doc):
#     ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
#     string = doc.page_content
#     if len([l for l in string if l.isalpha()]) < (len(string)//2):
#         return False
#     return True

# docs_split = [doc for doc in docs_split if include_doc(doc)]
print(len(docs_split))

34


In [6]:
for i in (0, 1, 2, 15, -1):
    print(f'[{i}] ', docs_split[i].page_content)
    print("-"*64)

[0]  MRKL Systems
A modular, neuro-symbolic architecture that combines large language
models, external knowledge sources and discrete reasoning
Ehud Karpas, Omri Abend, Yonatan Belinkov, Barak Lenz, Opher Lieber,
Nir Ratner, Yoav Shoham, Hoﬁt Bata, Yoav Levine, Kevin Leyton-Brown,
Dor Muhlgay, Noam Rozen, Erez Schwartz, Gal Shachaf, Shai
Shalev-Shwartz, Amnon Shashua, Moshe Tenenholtz
AI21 Labs
May 3, 2022
Abstract
Huge language models (LMs) have ushered in a new era for AI, serving as a gate-
way to natural-language-based knowledge tasks. Although an essential element of
modern AI, LMs are also inherently limited in a number of ways. We discuss these
limitations and how they can be avoided by adopting a systems approach. Con-
ceptualizing the challenge as one that involves knowledge and reasoning in addition
to linguistic processing, we deﬁne a ﬂexible architecture with multiple neural mod-
els, complemented by discrete knowledge and reasoning modules. We describe this
neuro-symbolic 

## The Document Summary

In [7]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from IPython.display import clear_output

In [8]:
class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are generating a running summary of the document. Make it readable by a technical user."
        " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
        " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
        " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    )), ('user', (
        "{format_instructions}. Follow the format precisely, including quotations and commas\n\n"
        "{info_base}\nWithout losing any of the info, update the knowledge base with the following: {input}"
    ))
])

In [9]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser
